In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

def load_data(file_path):
    return pd.read_csv(file_path)

def preprocess_data(df):
    le = LabelEncoder()
    df['event_type_encoded'] = le.fit_transform(df['event_type'])
    df['agent_id_encoded'] = le.fit_transform(df['agent_id'])
    df['context_encoded'] = le.fit_transform(df['context'])
    
    vocabularies = {
        'event_type': df['event_type'].unique().tolist(),
        'agent_id': df['agent_id'].unique().tolist(),
        'context': df['context'].unique().tolist()
    }
    
    return df, vocabularies

def create_cnn_model(input_shape):
    inputs = Input(shape=input_shape)
    
    x = Conv2D(64, (2, 2), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, (2, 2), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(128, (2, 2), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (2, 2), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)
    
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    return Model(inputs=inputs, outputs=x)

class HierarchicalModel:
    def __init__(self, input_shape, vocabularies):
        self.cnn = create_cnn_model(input_shape)
        self.vocabularies = vocabularies
        
        # Event type prediction
        event_input = Input(shape=(256,))
        event_output = Dense(len(vocabularies['event_type']), activation='softmax', name='event_type')(event_input)
        self.event_model = Model(inputs=event_input, outputs=event_output)
        
        # Agent ID prediction
        agent_input = Input(shape=(256 + len(vocabularies['event_type']),))
        agent_output = Dense(len(vocabularies['agent_id']), activation='softmax', name='agent_id')(agent_input)
        self.agent_model = Model(inputs=agent_input, outputs=agent_output)
        
        # Context prediction
        context_input = Input(shape=(256 + len(vocabularies['event_type']) + len(vocabularies['agent_id']),))
        context_output = Dense(len(vocabularies['context']), activation='softmax', name='context')(context_input)
        self.context_model = Model(inputs=context_input, outputs=context_output)
        
        self.compile_models()
    
    def compile_models(self):
        optimizer = Adam(learning_rate=0.001)
        self.event_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        self.agent_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        self.context_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    def fit(self, X, y, epochs=50, batch_size=64):
        cnn_features = self.cnn.predict(X)
        
        # Train event type model
        self.event_model.fit(cnn_features, y['event_type'], epochs=epochs, batch_size=batch_size, verbose=1)
        
        # Train agent ID model
        event_pred = self.event_model.predict(cnn_features)
        agent_input = np.concatenate([cnn_features, event_pred], axis=1)
        self.agent_model.fit(agent_input, y['agent_id'], epochs=epochs, batch_size=batch_size, verbose=1)
        
        # Train context model
        agent_pred = self.agent_model.predict(agent_input)
        context_input = np.concatenate([cnn_features, event_pred, agent_pred], axis=1)
        self.context_model.fit(context_input, y['context'], epochs=epochs, batch_size=batch_size, verbose=1)
    
    def predict_sequence(self, initial_input, sequence_length=5):
        predictions = []
        current_input = initial_input
        
        for _ in range(sequence_length):
            cnn_features = self.cnn.predict(current_input)
            
            event_pred = self.event_model.predict(cnn_features)
            agent_input = np.concatenate([cnn_features, event_pred], axis=1)
            agent_pred = self.agent_model.predict(agent_input)
            context_input = np.concatenate([cnn_features, event_pred, agent_pred], axis=1)
            context_pred = self.context_model.predict(context_input)
            
            pred = {
                'event_type': self.vocabularies['event_type'][np.argmax(event_pred[0])],
                'agent_id': self.vocabularies['agent_id'][np.argmax(agent_pred[0])],
                'context': self.vocabularies['context'][np.argmax(context_pred[0])]
            }
            predictions.append(pred)
            
            # Update the input for the next prediction
            current_input = self.update_input(current_input, pred)
        
        return predictions
    
    def update_input(self, current_input, prediction):
        # This method should update the input grid based on the new prediction
        # You'll need to implement the logic to update the grid based on your specific requirements
        # For now, we'll just return the current input as a placeholder
        return current_input

# Main pipeline
def process_event_log(file_path):
    # Load and preprocess data
    df = load_data(file_path)
    processed_df, grid, vocabularies = preprocess_data(df)
    
    # Prepare target variables
    y = {
        'event_type': pd.get_dummies(processed_df['event_type']).values,
        'agent_id': pd.get_dummies(processed_df['agent_id']).values,
        'context': pd.get_dummies(processed_df['context']).values
    }
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(grid, y, test_size=0.2, random_state=42)
    
    # Create and train hierarchical model
    model = HierarchicalModel((3, 3, 1), vocabularies)
    model.fit(X_train.reshape(-1, 3, 3, 1), y_train)
    
    # Make predictions
    initial_input = X_test[0:1].reshape(-1, 3, 3, 1)
    predictions = model.predict_sequence(initial_input, sequence_length=5)
    
    return predictions, X_test, y_test, vocabularies

# Run the pipeline
file_path = '10k_single_agent.csv'
predictions, X_test, y_test, vocabularies = process_event_log(file_path)

# Print the results
print("Predicted sequence:")
for i, pred in enumerate(predictions):
    print(f"Step {i+1}: {pred}")

ValueError: not enough values to unpack (expected 3, got 2)